# Individual Porject
# Formal modelling and statistical analysis of TOR

### Author: Leonidas Reppas
### Supervisor: Gethin Norman

#### Libraries:

In [1]:
import random

#### Relay statistics for the TOR network for 2009 (gathered from tor metrics):
* Roughly 1500 relays
* 625 exit relays
* less than 500,000 users

In [2]:
# Network setup / Relay creation

total_nodes = 0
non_exit_nodes = 0
exit_nodes = 0

class Relays():
    '''
    This class deals with the creation of relay types
    '''
    
    def create_non_exit_nodes(self, number):
        global non_exit_nodes
        non_exit_nodes += number
        
    def create_exit_nodes(self, number):
        global exit_nodes
        exit_nodes += number
        
    def calculate_total_nodes(self):
        global non_exit_nodes, exit_nodes, total_nodes
        total_nodes = exit_nodes + non_exit_nodes
        
    def print_nodes(self):
        global non_exit_nodes, exit_nodes, total_nodes
        print("""
        Number of non-exit nodes: {},
        Number of exit nodes: {},\n
        Number of total nodes in the network: {}
        """.format(non_exit_nodes, exit_nodes, total_nodes))
        
# Testing
relay = Relays()
relay.create_non_exit_nodes(875)
relay.create_exit_nodes(625)
relay.calculate_total_nodes()
relay.print_nodes()


        Number of non-exit nodes: 875,
        Number of exit nodes: 625,

        Number of total nodes in the network: 1500
        


#### Path selection algorithm implemented:
* Uniformly random, with no use of guard nodes. 
This is how Tor worked before 2004. The random choice lead to terrible bandwidth bottlenecks. Without the use of guard nodes,
every circuit had a probability to be compromised of (M/N)^2, where M is the amount of attacker controlled network resource
and N is the total network resource.

In [39]:
# Path Selection Algorithm
class TorPath():
    '''
    This class deals with the probabilistic path selection 
    algorithm for the different user models previously defined
    '''
    
    def select_path_typical(self, number):
        # first the exit node is selected
        # 91.5% of the exit flagged nodes
        # accept the ports of a typical user
        global exit_nodes, non_exit_nodes
        available_exit_nodes = exit_nodes * 0.92
        
        print("Typical User Model")
        for i in range(number):
            exit_choice = random.randrange(non_exit_nodes, non_exit_nodes + available_exit_nodes)
            middle_choice = random.randrange(1, non_exit_nodes)
            entry_choice = random.randrange(1, non_exit_nodes)
            
            # make sure no node is selected twice
            while (entry_choice == middle_choice):
                entry_choice = random.randrange(1, non_exit_nodes)
            
            print("""
            PATH #{}
            As an entry node we selected node #{},
            As a middle node we selected node #{},
            As an exit node we selected node #{}\n
            """.format(i+1, entry_choice, middle_choice, exit_choice))
            
    def select_path_irc(self, number):
        global exit_nodes, non_exit_nodes
        available_exit_nodes = exit_nodes * 0.8
        
        print("IRC User Model")
        for i in range(number):
            exit_choice = random.randrange(non_exit_nodes, non_exit_nodes + available_exit_nodes)
            middle_choice = random.randrange(1, non_exit_nodes)
            entry_choice = random.randrange(1, non_exit_nodes)
            
            while (entry_choice == middle_choice):
                entry_choice = random.randrange(1, non_exit_nodes)
            
            print("""
            PATH #{}
            As an entry node we selected node #{},
            As a middle node we selected node #{},
            As an exit node we selected node #{}\n
            """.format(i+1, entry_choice, middle_choice, exit_choice))
            
    def select_path_bittorrent(self, number):
        global exit_nodes, non_exit_nodes
        available_exit_nodes = exit_nodes * 0.2
        
        print("BitTorrent User Model")
        for i in range(number):
            exit_choice = random.randrange(non_exit_nodes, non_exit_nodes + available_exit_nodes)
            middle_choice = random.randrange(1, non_exit_nodes)
            entry_choice = random.randrange(1, non_exit_nodes)
            
            while (entry_choice == middle_choice):
                entry_choice = random.randrange(1, non_exit_nodes)
            
            print("""
            PATH #{}
            As an entry node we selected node #{},
            As a middle node we selected node #{},
            As an exit node we selected node #{}\n
            """.format(i+1, entry_choice, middle_choice, exit_choice))
        
        
torpath = TorPath()
torpath.select_path_typical(1)
torpath.select_path_irc(1)
torpath.select_path_bittorrent(1)

Typical User Model

            PATH #1
            As an entry node we selected node #279,
            As a middle node we selected node #194,
            As an exit node we selected node #1063

            
IRC User Model

            PATH #1
            As an entry node we selected node #729,
            As a middle node we selected node #396,
            As an exit node we selected node #1293

            
BitTorrent User Model

            PATH #1
            As an entry node we selected node #445,
            As a middle node we selected node #556,
            As an exit node we selected node #904

            
